In [25]:
# Read in the bikeshare data to a pandas dataframe just as you did for the previous assignment.
# Split the dataframe horizontally into two separate dataframes each having N/2 rows. 
# Call the dataframe with first N/2 rows bikeshare_train, and the latter half bikeshare_test
# Using the bikeshare_train dataframe run a regression with cnt as our target 
# variable, and temp, atemp, hum, workingday, hour, weathersit as the feature variables. 
# Specify the R^2 value. We'll call this our training model 
# The resulting model should have one more coefficient than the number of feature variables.
# Describe the error (in terms of R^2) between the model you just calculated and the bikeshare_test. 
# Is this higher or lower than the R^2 of the previous step? What's the reason for this difference?
# Create a binary variable for rush hour defined by 6-9am & 4-6pm.
# Run the regression again. Using a similar process used in steps 2 and 3, 
# assert whether this improves the results or not. Why? (You should be able to explain this quantitatively)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


%matplotlib inline

url = '../../../DAT_20_NYC/Data/bikeshare.csv'
# Read in the bikeshare data to a pandas dataframe just as you did for the previous assignment.
bikeshare_csv = pd.read_csv(url)

In [15]:
bikeshare_csv.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1


In [16]:
features = ['temp','atemp','hum','workingday','hr','weathersit']
# Split the dataframe horizontally into two separate dataframes each having N/2 rows. 
# Call the dataframe with first N/2 rows bikeshare_train, and the latter half bikeshare_test
bikeshare_train = bikeshare_csv[:(len(bikeshare_csv) / 2)]
bikeshare_test = bikeshare_csv[(len(bikeshare_csv) / 2):]

print "Original DF: %s" % len(bikeshare_csv) + " Rows"
print "Train: %s" % len(bikeshare_train) + " Rows"
print "Test: %s" % len(bikeshare_test) + " Rows"

Original DF: 17379 Rows
Train: 8689 Rows
Test: 8690 Rows


In [21]:
# Using the bikeshare_train dataframe run a regression with cnt as our target 
# variable, and temp, atemp, hum, workingday, hour, weathersit as the feature variables. 

import statsmodels.formula.api as smf

# create a fitted model in one line
model = smf.ols(formula='cnt ~ temp + atemp + hum + workingday + hr + weathersit', data=bikeshare_train).fit()

# Specify the R^2 value. We'll call this our training model 
print "R^2 is: %s" % model.rsquared

# The resulting model should have one more coefficient than the number of feature variables.
model.summary()

R^2 is: 0.36711033766


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     839.3
Date:                Thu, 07 May 2015   Prob (F-statistic):               0.00
Time:                        18:09:00   Log-Likelihood:                -52873.
No. Observations:                8689   AIC:                         1.058e+05
Df Residuals:                    8682   BIC:                         1.058e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     24.4288      5.975      4.089      0.000        12.717    36.140
temp         137.3518     46.304      2.966      0.003        46.585   228.119
atemp        153.7565     51.876      2.964      0.003        52.067   255.446
hum         -127.7197      6.686    -19.101      0.000      -140.827  -114.613
workingday    -1.0739      2.457     -0.437      0.662        -5.891     3.743
hr             6.0314      0.173     34.895      0.000         5.693     6.370
weathersit    -4.7972      1.944     -2.467      0.014        -8.608    -0.986
==============================================================================
Omnibus:                     1474.307   Durbin-Watson:                   0.550
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2499.956
Skew:                           1.118   Prob(JB):                         0.00
Kurtosis:                       4.380   Cond. No.                         827.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
# Describe the error (in terms of R^2) between the model you just calculated and the bikeshare_test. 
model2 = smf.ols(formula='cnt ~ temp + atemp + hum + workingday + hr + weathersit', data=bikeshare_test).fit()
print "R^2 is: %s" % model2.rsquared
# Is this higher or lower than the R^2 of the previous step? What's the reason for this difference?


R^2 is: 0.339247807659


R2 is lower here, probably because this model is a poor predictor. 

In [28]:
# Create a binary variable for rush hour defined by 6-9am & 4-6pm.
# Run the regression again. Using a similar process used in steps 2 and 3, 
# assert whether this improves the results or not. Why? (You should be able to explain this quantitatively
from math import ceil
rush_hour = [int((x > 5 and x < 10) or (x > 15 and x < 19)) for x in bikeshare_csv['hr']]

bikeshare_train['rush_hour'] = rush_hour[ : 8689]
bikeshare_test['rush_hour'] = rush_hour[8689 : ]

model3 = smf.ols(formula='cnt ~ temp + atemp + hum + workingday + hr + weathersit + rush_hour', data=bikeshare_train).fit()
model4 = smf.ols(formula='cnt ~ temp + atemp + hum + workingday + hr + weathersit + rush_hour', data=bikeshare_test).fit()

print "Bikeshare 'train' with rush hour R^2 is: %s" % model3.rsquared
print "Bikeshare 'test' with rush hour R^2 is: %s" % model4.rsquared
print "model3 p values: %s" % model3.pvalues
print "model4 p values: %s" % model4.pvalues

print model3.summary()
print model4.summary()

Bikeshare 'train' with rush hour R^2 is: 0.505642825624
Bikeshare 'test' with rush hour R^2 is: 0.514960344161
model3 p values: Intercept     5.879179e-02
temp          2.401363e-03
atemp         2.248664e-04
hum           5.791844e-91
workingday    6.341201e-01
hr            0.000000e+00
weathersit    4.026720e-05
rush_hour     0.000000e+00
dtype: float64
model4 p values: Intercept      5.106513e-03
temp           1.641426e-01
atemp          1.069424e-11
hum           1.987064e-134
workingday     8.769749e-04
hr            7.654072e-291
weathersit     4.651364e-02
rush_hour      0.000000e+00
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.505
Method:                 Least Squares   F-statistic:                     1268.
Date:                Thu, 07 May 2015   Prob (F-statistic):          

/Users/corey/.virtualenvs/GA/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/corey/.virtualenvs/GA/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# assert whether this improves the results or not. Why? (You should be able to explain this quantitatively)

If I'm interpreting the results correctly, rush_hour has a p-value of zero in both sets, which indicates that it's a nearly perfect predictor of the bikeshare data? Including that as a predictor would dramatically raise the R^2 on both models. 